In [138]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import date
import plotly.graph_objects as go

In [139]:
c_to_f = lambda c :(9/5)*c + 32

In [140]:
#read in .xls to df
df=pd.read_excel("n_obs/SIO_TEMP_20230501.xls",skiprows=47, header=None) #change excel filename
df = df.rename(columns={0: 'year',
                        1: 'month',
                        2: 'day',
                        5: 'sst'})
df = df[(df['year'] > 1916) & (df['year'] < 2023)] #exclude partial years 1916 & 2023
df['date'] = pd.to_datetime(df[['year','month','day']])
df

,year,month,day,3,4,sst,6,7,8,date
114,1917,1,1,NaN,NaN,NaN,0,NaN,NaN,1917-01-01
115,1917,1,2,NaN,NaN,11.5,0,NaN,NaN,1917-01-02
116,1917,1,3,NaN,NaN,11.9,0,NaN,NaN,1917-01-03
117,1917,1,4,NaN,NaN,11.5,0,NaN,NaN,1917-01-04
118,1917,1,5,NaN,NaN,11.7,0,NaN,NaN,1917-01-05
...,...,...,...,...,...,...,...,...,...,...
38825,2022,12,27,1440.0,0.0,14.7,0,14.6,0.0,2022-12-27
38826,2022,12,28,1320.0,0.0,15.1,0,14.9,0.0,2022-12-28
38827,2022,12,29,1430.0,0.0,14.9,0,14.9,0.0,2022-12-29
38828,2022,12,30,1719.0,0.0,15.3,0,15.1,0.0,2022-12-30


In [141]:
#make annual climatology and long-term mean
climatology = df.groupby(['year']).mean().reset_index()
long_term_mean = df.sst.mean()
X = climatology.year.values.reshape(-1,1)
Y = climatology.sst.values.reshape(-1,1)
lm = LinearRegression()
lm.fit(X,Y)  # perform linear regression
century_trend = (lm.coef_[0] * 100)[0]

In [142]:
climatology.sst

0      16.742715
1      17.385113
2      16.810032
3      16.576177
4      16.536223
         ...    
101    19.082581
102    18.681795
103    18.447528
104    17.849972
105    18.289863
Name: sst, Length: 106, dtype: float64

In [149]:
fig = go.Figure()

coldest_cold = climatology.sst.min()
warmest_warm = climatology.sst.max()

def get_color(sst):
    """Interpolate between blue, white, and red based on parameter t."""
    t = (climatology.sst.max()-sst)/(climatology.sst.max() - climatology.sst.min())
    blue = (200, 0, 0) #TODO unflip these
    white = (255, 255, 255)
    red = (0, 0, 200)
    # Ensure t is within [0, 1]
    if t <= 0.5:
        # Interpolate between blue and white
        interpolated_color = tuple(int(c1 + (c2 - c1) * t * 2) for c1, c2 in zip(blue, white))
    else:
        # Interpolate between white and red
        t = (t - 0.5) * 2
        interpolated_color = tuple(int(c1 + (c2 - c1) * t) for c1, c2 in zip(white, red))

    return 'rgb('+str(interpolated_color[0])+', '+str(interpolated_color[1])+', '+str(interpolated_color[2])+')'

for i, sst in enumerate(climatology.sst):
    i += 1917
    df_year = df[(df['year'] == i)]
    year_warmest_sst = df_year.sst.max()
    year_warmest_day = df_year[df_year.sst == year_warmest_sst].date.iloc[0].strftime('%b %d')
    year_coldest_sst = df_year.sst.min()
    year_coldest_day = df_year[df_year.sst == year_coldest_sst].date.iloc[0].strftime('%b %d')
    fig.add_shape(
        type="rect",
        x0=i,
        y0=0,
        x1=i+1,
        y1=30,
        layer="below",
        line=dict(color="rgba(0,0,0,0)"),
        fillcolor=get_color(sst))
    '''
    fig.add_trace(go.Scatter(
    x=[i], #TODO fix so they're not just plotted for each year but on their actual date
    y=[year_warmest_sst],
    mode='markers',
    marker = dict(color='black',size=2),
    text = 'Warmest Day: '+str(year_warmest_sst)+'°C on '+year_warmest_day,
    hoverinfo='text',
    showlegend=False))

    fig.add_trace(go.Scatter(
    x=[i],
    y=[year_coldest_sst],
    mode='markers',
    marker = dict(color='black',size=2),
    text = 'Coldest Day: '+str(year_coldest_sst)+'°C on '+year_coldest_day,
    hoverinfo='text',
    showlegend=False))
    '''
    
fig.add_trace(go.Scatter(
    x=df.groupby('year')['year'].first(),
    y=climatology.sst,
    name = 'Yearly average',
    mode='lines',
    line=dict(
        width=5,
        color='white'),
    text=[f'Year: {year}<br>Average: {temp:.1f}°C' for year, temp in zip(df.groupby('year')['year'].first(), climatology.sst)],
    hoverinfo='text',
    showlegend=False))

fig.update_layout(
    title='SST at Scripps Pier',
    xaxis=dict(title='Year', range=[climatology.year.min(), climatology.year.max()], showgrid=False),
    yaxis=dict(title='Sea Surface Temperature (°C)', range=[climatology.sst.min() - 1, climatology.sst.max() + 1], showgrid=False),
    hovermode='x unified',
    plot_bgcolor='white')

fig.show()

In [144]:
climatology.year.min()

1917